In [ ]:
# !git clone https://github.com/AshishJangra27/Face-Generator-with-GAN

import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

import imageio
from tqdm import tqdm


os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

generator = tf.keras.models.load_model('/kaggle/working/Face-Generator-with-GAN/generator_700.h5', compile=False)

### 1. Generate Images

In [ ]:
noise = tf.random.normal([1, 100])

with tf.device('/CPU:0'):
    generated_images = generator(noise, training=False)

generated_images = (generated_images + 1) / 2.0

plt.imshow(generated_images[0])
plt.axis('off')

### 2. Bulk Generation

In [ ]:
def generate_multiple_images(model, noise_dim=100, num_images=10):
    fig, axes = plt.subplots(2, 5, figsize=(15, 7))  # 10 rows, 5 columns

    for i in range(num_images):
        noise = tf.random.normal([1, noise_dim])  # Generate random noise

        with tf.device('/CPU:0'):
            generated_image = model(noise, training=False)

        generated_image = (generated_image + 1) / 2.0  # Rescale for visualization

        # Plot the generated image
        ax = axes[i // 5, i % 5]  # Row-wise placement
        ax.imshow(generated_image[0])
        ax.axis('off')
        ax.set_title(f"Image {i+1}", fontsize=12, pad=10)

    plt.subplots_adjust(wspace=0.3, hspace=0.3)  # Adjust spacing between images
    plt.show()

generate_multiple_images(generator)

### 3. Getting Classification on Faces

In [ ]:
classifier = tf.keras.models.load_model('/kaggle/input/gender-classifier-mobilenet/keras/gender-classifier-mobilenet/1/gender_classifier.keras')

noise = tf.random.normal([1, 100])

generated_image = generator(noise, training=False)
generated_image = (generated_image + 1) / 2.0  # Rescale from [-1,1] to [0,1]

generate_img = tf.image.resize(generated_image, (224, 224))

prediction = classifier.predict(generate_img, verbose = 0)
label = "Male" if prediction[0][0] > 0.5 else "Female" 

plt.title(label + str(prediction))
plt.imshow(generated_image[0])
plt.axis('off')
plt.show()

### 4. Checking Images and Predictions

In [ ]:
fig, axes = plt.subplots(5, 10, figsize=(15, 7))  # 5 rows, 10 columns
fig.subplots_adjust(hspace=0.5)  # Adjust spacing

for i in range(50):
    noise = tf.random.normal([1, 100])
    generated_image = generator(noise, training=False)
    generated_image = (generated_image + 1) / 2.0  # Rescale to [0,1]

    resized_img = tf.image.resize(generated_image, (224, 224))
    resized_img = tf.expand_dims(resized_img[0], axis=0)  # Ensure batch dimension

    prediction = classifier.predict(resized_img, verbose=0)
    
    prob = prediction[0][0]

    # Classification with new conditions
    if prob > 0.95:
        label = "Male"
    elif prob < 0.05:
        label = "Female"
    else:
        label = "IDK"

    row, col = divmod(i, 10)  # Calculate grid position
    axes[row, col].imshow(generated_image[0])
    axes[row, col].set_title(label, fontsize=8)
    axes[row, col].axis('off')

plt.show()

### 5. Checking High Quality Male Images

In [ ]:
male_images = []
pbar = tqdm(total=10, desc="Generating High-Confidence Male Images")  # Progress Bar

# Generate images until we have 10 males with high confidence
while len(male_images) < 10:
    noise = tf.random.normal([1, 100])
    generated_image = generator(noise, training=False)
    generated_image = (generated_image + 1) / 2.0  # Rescale to [0,1]

    resized_img = tf.image.resize(generated_image, (224, 224))
    resized_img = tf.expand_dims(resized_img[0], axis=0)  # Ensure batch dimension

    prediction = classifier.predict(resized_img, verbose=0)
    prob = prediction[0][0]

    # Keep only males with high confidence (>0.99)
    if prob > 0.99:
        male_images.append((generated_image[0], prob))  # Store image & confidence
        pbar.update(1)  # Update progress bar

pbar.close()  # Close progress bar when done

# Plot the 10 high-confidence male images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))  # 2 rows, 5 columns
fig.subplots_adjust(hspace=0.5)

for i, (img, prob) in enumerate(male_images):
    row, col = divmod(i, 5)
    axes[row, col].imshow(img)
    axes[row, col].set_title(f"Male ({prob:.2f})", fontsize=10)
    axes[row, col].axis('off')

plt.show()


### 6. Generating High Quality Male Images

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image

output_folder = "generated_males"
os.makedirs(output_folder, exist_ok=True)

# Initialize CSV data storage
data = []

# Initialize progress bar
num_images = 1000
pbar = tqdm(total=num_images, desc="Generating High-Confidence Male Images")

male_count = 0

# Generate images until we have 1000 males with high confidence
while male_count < num_images:
    noise = tf.random.normal([1, 100])  # Generate noise vector
    generated_image = generator(noise, training=False)
    generated_image = (generated_image + 1) / 2.0  # Rescale to [0,1]

    resized_img = tf.image.resize(generated_image, (224, 224))
    resized_img = tf.expand_dims(resized_img[0], axis=0)  # Ensure batch dimension

    prediction = classifier.predict(resized_img, verbose=0)
    prob = prediction[0][0]

    # Keep only males with high confidence (>0.99)
    if prob > 0.99:
        file_path = os.path.join(output_folder, f"male_{male_count:04d}.png")
        
        # Convert Tensor to Image and Save
        img_array = (generated_image[0].numpy() * 255).astype(np.uint8)
        img = Image.fromarray(img_array)
        img.save(file_path)

        # Store noise vector and file path in data list
        data.append([file_path, noise.numpy().tolist()])

        male_count += 1  # Increment count
        pbar.update(1)  # Update progress bar

pbar.close()  # Close progress bar

# Save noise vectors and image paths in a CSV file
df = pd.DataFrame(data, columns=["file_path", "noise_vector"])
csv_path = "male_images_data.csv"
df.to_csv(csv_path, index=False)

print(f"✅ Successfully generated {num_images} male images.")
print(f"📁 Images saved in: {output_folder}")
print(f"📄 CSV file saved as: {csv_path}")

### 7. Generating High Quality Female Images

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image

output_folder = "generated_females"
os.makedirs(output_folder, exist_ok=True)

# Initialize CSV data storage
data = []

# Initialize progress bar
num_images = 1000
pbar = tqdm(total=num_images, desc="Generating High-Confidence Female Images")

male_count = 0


while male_count < num_images:
    noise = tf.random.normal([1, 100])  # Generate noise vector
    generated_image = generator(noise, training=False)
    generated_image = (generated_image + 1) / 2.0  # Rescale to [0,1]

    resized_img = tf.image.resize(generated_image, (224, 224))
    resized_img = tf.expand_dims(resized_img[0], axis=0)  # Ensure batch dimension

    prediction = classifier.predict(resized_img, verbose=0)
    prob = prediction[0][0]

    # Keep only female with high confidence (>0.01)
    if prob < 0.01:
        file_path = os.path.join(output_folder, f"female_{male_count:04d}.png")
        
        # Convert Tensor to Image and Save
        img_array = (generated_image[0].numpy() * 255).astype(np.uint8)
        img = Image.fromarray(img_array)
        img.save(file_path)

        # Store noise vector and file path in data list
        data.append([file_path, noise.numpy().tolist()])

        male_count += 1  # Increment count
        pbar.update(1)  # Update progress bar

pbar.close()  # Close progress bar

# Save noise vectors and image paths in a CSV file
df = pd.DataFrame(data, columns=["file_path", "noise_vector"])
csv_path = "female_images_data.csv"
df.to_csv(csv_path, index=False)

print(f"✅ Successfully generated {num_images} female images.")
print(f"📁 Images saved in: {output_folder}")
print(f"📄 CSV file saved as: {csv_path}")


### 8. Finding Gender Vector

In [ ]:
import ast

male_vec   = pd.read_csv('/kaggle/working/male_images_data.csv')['noise_vector']
female_vec = pd.read_csv('/kaggle/working/female_images_data.csv')['noise_vector']

male_vec = np.array([np.array(ast.literal_eval(vec)) for vec in male_vec])
female_vec = np.array([np.array(ast.literal_eval(vec)) for vec in female_vec])

avg_male_vec = np.mean(male_vec, axis=0)
avg_female_vec = np.mean(female_vec, axis=0)

print("Average Noise Vector (Male):", avg_male_vec)
print("Average Noise Vector (Female):", avg_female_vec)

### 9. Display Vectors

In [ ]:
def generate_and_predict(noise_vec):
    generated_img = generator(noise_vec, training=False)
    generated_img = (generated_img + 1) / 2.0  # Rescale from [-1,1] to [0,1]
    resized_img = tf.image.resize(generated_img, (224, 224))
    prediction = classifier.predict(resized_img, verbose=0)
    label = "Male" if prediction[0][0] > 0.5 else "Female"
    return generated_img[0], f"{label} ({prediction[0][0]:.2f})"

male_img, male_label = generate_and_predict(avg_male_vec)
female_img, female_label = generate_and_predict(avg_female_vec)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(male_img)
axes[0].set_title(male_label)
axes[0].axis("off")

axes[1].imshow(female_img)
axes[1].set_title(female_label)
axes[1].axis("off")

plt.show()

### 10. Getting Gender Vector

In [ ]:
gender_vec = avg_male_vec - avg_female_vec

### 11. Testing Gender Style Transfer

#### 11.1) Checking different variations of same image

In [ ]:
def generate(noise):
    return (generator(noise, training=False) + 1) / 2.0 

noise = tf.random.normal([1, 100])
modifications = [2, 1.5, 1, 0.5, 0, -0.5, -1, -1.5, -2]

fig, axes = plt.subplots(1, 9, figsize=(15, 3))

for i, mod in enumerate(modifications):
    axes[i].imshow(generate(noise + mod * gender_vec)[0])
    axes[i].axis("off")

plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout(pad=0)
plt.show()

#### 11.2) Checking different variations on different image

In [ ]:
def generate(noise):
    return (generator(noise, training=False) + 1) / 2.0  # Rescale [-1,1] to [0,1]

# Generate 10 different noise vectors
noises = tf.random.normal([10, 100])
modifications = [2.5,2, 1.5, 1, 0.5, 0, -0.5, -1, -1.5, -2,-2.5]

fig, axes = plt.subplots(10, 11, figsize=(15, 15))  # 10 rows, 11 columns

for row, noise in enumerate(noises):
    for col, mod in enumerate(modifications):
        axes[row, col].imshow(generate(noise + mod * gender_vec)[0])
        axes[row, col].axis("off")

plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout(pad=0)
plt.show()

#### 11.3) Checking different bulk variations on bulk image

In [ ]:
def generate(noise):
    return (generator(noise, training=False) + 1) / 2.0  # Rescale from [-1,1] to [0,1]

noises = tf.random.normal([20, 100])

modifications = np.linspace(-3, 3, 30)  # 30 steps from -3 to 3

fig, axes = plt.subplots(20, 30, figsize=(30, 20))

for row, noise in enumerate(noises):
    for col, mod in enumerate(modifications):
        axes[row, col].imshow(generate(noise + mod * gender_vec)[0])
        axes[row, col].axis("off")

plt.subplots_adjust(wspace=0, hspace=0)  # Remove spaces
plt.tight_layout(pad=0) 
plt.show()

#### 11.4) Variations of an Image with GIF

In [ ]:
def generate(noise):
    return (generator(noise, training=False) + 1) / 2.0  # Rescale [-1,1] to [0,1]

# Start with a single noise vector
noise = tf.random.normal([1, 100])

# Generate 30 modifications smoothly transitioning
modifications = np.linspace(-3, 3, 60)

# Create list to store frames
frames = []

# Generate each variant and store in frames
for mod in modifications:
    img = generate(noise + mod * gender_vec)[0].numpy()  # Convert tensor to NumPy array
    frames.append((img * 255).astype(np.uint8))  # Convert to uint8 for GIF

imageio.mimsave("face_transformation.gif", frames, duration=5/30)  # 30 FPS over 5s

# Display GIF inside a Jupyter Notebook (Optional)
from IPython.display import display, Image
display(Image(filename="face_transformation.gif"))

#### 11.5) Giant GIF

In [ ]:
import tensorflow as tf
import numpy as np
import imageio
from tqdm import tqdm

def generate(noise):
    return (generator(noise, training=False) + 1) / 2.0  # Rescale [-1,1] to [0,1]

# Generate 25 different noise vectors (5x5 grid)
noises = tf.random.normal([25, 100])

# Generate 60 smooth modifications
modifications = np.linspace(-3, 3, 60)

frames = []

for mod in tqdm(modifications, desc="Generating Frames"):
    grid_images = []  # Store rows of the 5x5 grid

    for i in range(5):  # 5 rows
        row_images = [generate(noises[i * 5 + j] + mod * gender_vec)[0].numpy() for j in range(5)]  # 5 columns
        grid_images.append(np.hstack(row_images))  # Stack row horizontally

    stacked_frame = np.vstack(grid_images)  # Stack all rows vertically
    frames.append((stacked_frame * 255).astype(np.uint8))  # Convert to uint8 for GIF

# Save final 5×5 animated GIF (60 frames, 5 seconds total)
imageio.mimsave("small_grid_face_transformation.gif", frames, duration=5/60)  # 60 FPS for smooth animation

# Display GIF in Jupyter Notebook (Optional)
from IPython.display import display, Image
display(Image(filename="small_grid_face_transformation.gif"))

### 12. Saving Images

In [ ]:
import shutil

# Define folder paths
male_folder = "/kaggle/working/generated_males"
female_folder = "/kaggle/working/generated_females"

# Define output zip filenames
male_zip = "/kaggle/working/male_images"
female_zip = "/kaggle/working/female_images"

# Zip the Male folder
shutil.make_archive(male_zip, 'zip', male_folder)

# Zip the Female folder
shutil.make_archive(female_zip, 'zip', female_folder)

print("Zipping complete! Files saved as male_images.zip and female_images.zip")
